In [74]:
# Import modules
import pandas as pd
import numpy as np
import sklearn
import torch
from torch import nn
from sklearn.model_selection import train_test_split
torch.__version__

'2.2.2'

In [75]:
# Load the dataset and review the first 100 characters
with open("the-verdict.txt", "r",encoding="utf-8") as f:
    raw_text = f.read()
print("Length of text:", len(raw_text))
print(raw_text[:9])

Length of text: 20479
I HAD alw


In [76]:
1+1

2

In [77]:
data = [1,2,3]
tens1 = torch.tensor(data)
print(tens1)

tensor([1, 2, 3])


In [78]:
array = np.array([1,2,3])
print(array)
tens2 = torch.from_numpy(array)
print(tens2)

[1 2 3]
tensor([1, 2, 3])


In [79]:
# Ch 2, p 22 - split text
import re
text = "Hello, world!  This, is a test."
result = re.split(r'([!,.]|\s)', text)
print(result)
result2 = [item for item in result if item.strip()]
print(result2)

['Hello', ',', '', ' ', 'world', '!', '', ' ', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']
['Hello', ',', 'world', '!', 'This', ',', 'is', 'a', 'test', '.']


In [80]:
# Add more puncuation and  --- to the split
text = "Hello, world!  This: -- is a test. Maybe?"
result = re.split(r'([!,.?:]|--|\s)', text)
print(result)
result2 = [item for item in result if item.strip()]
print(result2)

['Hello', ',', '', ' ', 'world', '!', '', ' ', '', ' ', 'This', ':', '', ' ', '', '--', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '', ' ', 'Maybe', '?', '']
['Hello', ',', 'world', '!', 'This', ':', '--', 'is', 'a', 'test', '.', 'Maybe', '?']


In [155]:
# p24 - Use tokenizer scheme on Wharton text
preprocessed = re.split(r'([!,.?:;"_()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))


4690


In [156]:
# Test output of tokenized used on The Verdict  
print(preprocessed[0:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [157]:
# 2.3 - Create a vocabulary

# Create list of all uniuqe words and sort
all_words   = sorted(set(preprocessed))
vocab_size  = len(all_words)
print(vocab_size)

1130


In [158]:
# Use vocab to create a dictionary
vocab = {token: integer for integer, token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i > 20:
        break
    

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)


In [159]:
# p 27 - Make simple tokenizer Class
class SimpleTokenV1:
    
    def __init__(self, vocab):
        self.str_to_int = vocab # stores the vocab dictionary
        self.int_to_str = {integer: string for string, integer in vocab.items()} # reverse dictionary (token IDS to text tokens)

    def encode(self, text):
        preprocessed = re.split(r'([!,.?:;"()\']|--|\s)', text)
        preprocessed = [item for item in preprocessed if item.strip()]
        ids = [self.str_to_int[token] for token in preprocessed]
        return ids # return the token IDs

    def decoder(self, ids):  # Converts token IDs back to text
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([!,.?:;"_()\'])', r'\1', text)
        return text
    

In [161]:
# Test the tokenizer
tokenizer = SimpleTokenV1(vocab)
text = """"It's the last he painted, you know," 
Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [162]:
# Test the decoder
print(tokenizer.decoder(ids))

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


In [163]:
# Need to account for words not in the vocab
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>","<|unk|>"])
vocab = {token: integer for integer, token in enumerate(all_tokens)}
print(len(vocab.items()))

1132


In [164]:
# Confirm that new characters are in the vocab
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [165]:
# p 31 - Adjust tokenizer class to reflect UNK token
class SimpleTokenizerV2:
    
    def __init__(self, vocab):
        self.str_to_int = vocab # stores the vocab dictionary
        self.int_to_str = {integer: string for string, integer in vocab.items()} # reverse dictionary (token IDS to text tokens)

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]

        # Account for words not in the vocab
        preprocessed = [token if token in self.str_to_int else "<|unk|>" for token in preprocessed]

        ids = [self.str_to_int[token] for token in preprocessed]
        return ids # return the token IDs

    def decoder(self, ids):  # Converts token IDs back to text
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [166]:
# Test the tokenizer adjusted for unknown words
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [167]:
# Get token IDs
tokenizer = SimpleTokenizerV2(vocab)
print(tokenizer.encode(text))

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]


In [168]:
# Test the decoder
print(tokenizer.decoder(tokenizer.encode(text)))

<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


In [94]:
# 2.5 - p 33 - Byte Pair Encoding

In [95]:
from importlib.metadata import version
import tiktoken
print("Tiktoken version:", version("Tiktoken"))

Tiktoken version: 0.8.0


In [130]:
# Instantiate BPE tokenizer
tokenizer = tiktoken.get_encoding("gpt2")

In [131]:
# Encode text to integers
text = ("Hello, do you like tea? <|endoftext]> In the sunlit terraces " 
        "of someunknownPlace.")
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 1279, 91, 437, 1659, 5239, 60, 29, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 27271, 13]


In [132]:
# Decode integers back to text
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext]> In the sunlit terraces of someunknownPlace.


In [133]:
# Use BPE on The Verdict
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))
print(type(enc_text))

5145
<class 'list'>


In [134]:
# Take sample of encoded text
enc_sample = enc_text[50:]
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"x:  {x}")
print(f"y:      {y}")

x:  [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [135]:
#  Test print integers
for i in range(1,context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "------->", desired)

[290] -------> 4920
[290, 4920] -------> 2241
[290, 4920, 2241] -------> 287
[290, 4920, 2241, 287] -------> 257


In [136]:
# Convert back to text using decoder
for i in range(1,context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "------->", tokenizer.decode([desired]))

 and ------->  established
 and established ------->  himself
 and established himself ------->  in
 and established himself in ------->  a


In [138]:
# Build dataset Class   
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt) # Tokenize the entire text
        
        # Uses overlapping chunks of max_length tokens
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk)) 

    # Returns total number of rows in the dataset
    def __len__(self):
        return len(self.input_ids)
    
    # Returns a single row of the dataset
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [139]:
# Dataloader

def create_dataloader_v1(txt,batch_size=4,max_length=256,stride=128, shuffle=True, drop_last=True, num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2") # Instantiate BPE tokenizer
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride) # Creates dataset
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                              shuffle=shuffle, drop_last=drop_last, # drop_last=True to drop the last incomplete batch
                                                num_workers=num_workers) # num_workers=0 to use single process
    return dataloader

In [140]:
# Test dataloader with batch size of 1

dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [141]:
# Fetch another batch
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [142]:
# Test with larger batch size and stride, so no overlap
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs,targets = next(data_iter)
print("Inputs:\n",inputs)
print("\nTargets:\n",targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


In [143]:
# 2.7 (p41) - Create token embeddings from token IDs

# Sample input layer
input_ids = torch.tensor([2,3,5,1])

vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

print(embedding_layer(input_ids))

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)
tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


In [144]:
# 2.8 (p43) - Need to add posistional encoding to the token id embeddings
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)


In [145]:
# Example
max_length = 4
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=max_length, stride=max_length, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape) 
print(type(inputs)) 
print(type(targets)) 

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])
<class 'torch.Tensor'>
<class 'torch.Tensor'>


In [147]:
# Embed token IDs into 256-dimensional vectors
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [152]:
# Absolute embedding position
context_length = max_length
pos_embedding_layer = nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)
print(pos_embeddings)


torch.Size([4, 256])
tensor([[ 1.9192, -0.0239, -1.3632,  ...,  1.0244,  0.6319, -0.2026],
        [ 1.0580, -0.1855,  0.5553,  ...,  0.0593, -1.2205,  0.7917],
        [ 0.3932,  0.1786, -0.4253,  ...,  1.4493, -0.8674,  0.8658],
        [ 0.4966, -1.9540, -1.0235,  ...,  1.1835,  1.0848,  0.1064]],
       grad_fn=<EmbeddingBackward0>)


In [149]:
# Combine token and positional embeddings
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)
#print(input_embeddings)

torch.Size([8, 4, 256])
